In [ ]:
# import packages 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib, time
import cvxpy as cp

In [ ]:
# function definitions
def compute_stats(A, y, K):
    # This function computes the class means and covariance matrices.
    class_means = np.zeros((A.shape[1],K))
    covariances = np.zeros((A.shape[1],A.shape[1],K))
    
    for k in range(K):
        class_means[:,k] = np.mean(A[y[:,0] == k, :], axis=0)
        A_minus_mean_k = A[y[:,0] == k, :] - class_means[:,k]
        covariances[:,:,k] = np.matmul(A_minus_mean_k.T, A_minus_mean_k) # / (np.sum(y[:,0] == k)-1)
    
    return class_means, covariances
def compute_acc_binary(projected_data_train, labels_train, projected_data_test, labels_test, return_pred=False):
    class_mean_proj_0 = np.mean(projected_data_train[labels_train[:,0] == 0])
    class_mean_proj_1 = np.mean(projected_data_train[labels_train[:,0] == 1])
    class_means_proj = np.array([class_mean_proj_0, class_mean_proj_1])
    
    ypred = np.argmin(np.abs(projected_data_test - class_means_proj), axis=1)
    
    acc = np.sum(ypred == labels_test[:, 0]) / labels_test.shape[0]
    
    if return_pred:
        return acc, np.expand_dims(ypred, axis=1)
    else:
        return acc
def check_if_already_exists(element_list, element):
    # check if element exists in element_list
    # where element is a numpy array
    for i in range(len(element_list)):
        if np.array_equal(element_list[i], element):
            return True
    return False
def generate_sign_patterns(A, P, verbose=False): 
    # generate sign patterns
    n, d = A.shape
    unique_sign_pattern_list = []  # sign patterns
    h_vector_list = []             # random vectors used to generate the sign paterns
    np.random.seed(0)
    
    while len(unique_sign_pattern_list) != P:
        # obtain a sign pattern
        h = np.random.normal(0, 1, (d,1)) # sample h
        sampled_sign_pattern = (np.matmul(A, h) >= 0)[:,0]
        # check whether that sign pattern has already been used
        if not check_if_already_exists(unique_sign_pattern_list, sampled_sign_pattern):
            unique_sign_pattern_list.append(sampled_sign_pattern)
            h_vector_list.append(h)

    return unique_sign_pattern_list, h_vector_list

In [ ]:
# two spirals dataset
import math
def spiral_xy(i, spiral_num):
    """
    Create the data for a spiral.

    Arguments:
        i runs from 0 to 96
        spiral_num is 1 or -1
    """
    φ = i/16 * math.pi
    r = 6.5 * ((104 - i)/104)
    x = (r * math.cos(φ) * spiral_num)/13 + 0.5
    y = (r * math.sin(φ) * spiral_num)/13 + 0.5
    return (x, y)
def spiral(spiral_num, n):
    return [spiral_xy(i, spiral_num) for i in range(n)]

n_ = 60

a = np.array(spiral(1,n_))
b = np.array(spiral(-1,n_))

A = np.concatenate((a,b), axis=0)
y = np.concatenate((np.zeros((a.shape[0],1)), np.ones((b.shape[0],1))), axis=0)

np.random.seed(0)
permute = np.random.choice(A.shape[0], A.shape[0], replace=False)
A_test = A[permute[:A.shape[0]//200], :]
y_test = y[permute[:y.shape[0]//200], :]

A = A[permute[A.shape[0]//200:], :]
y = y[permute[y.shape[0]//200:], :]

n, d = A.shape
print(A.shape, y.shape, A_test.shape, y_test.shape)


fig = matplotlib.pyplot.gcf()
fig.set_size_inches(6, 6)

plt.rcParams.update({'font.size': 18})
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')

# plt.plot(a[:, 0], a[:, 1], 'x', label="Class 0")
# plt.plot(b[:, 0], b[:, 1], 'o', label="Class 1")

plt.plot(A[y[:,0]==0, 0], A[y[:,0]==0, 1], 'x', label="Class 0")
plt.plot(A[y[:,0]==1, 0], A[y[:,0]==1, 1], 'x', label="Class 1")

plt.legend()
plt.grid()
# plt.savefig('figures/two_spirals_dataset.eps', bbox_inches="tight")

In [ ]:
# column of ones
A = np.concatenate((A, np.ones((A.shape[0],1))), axis=1)
n, d = A.shape
print(n, d)

A_test = np.concatenate((A_test, np.ones((A_test.shape[0],1))), axis=1)

### FLDA

In [ ]:
# FLDA
class_means, covariances = compute_stats(A, y, 2)
print(class_means.shape, covariances.shape)

beta = 0.1
a = np.matmul(np.linalg.inv(covariances[:,:,0] + covariances[:,:,1] + beta*np.eye(class_means.shape[0])), class_means[:,0:1]-class_means[:,1:2])

In [ ]:
projected = np.matmul(A, a)

In [ ]:
# histogram
plt.rcParams.update({'font.size': 18})
plt.hist(projected[y[:,0] == 0], bins=10, alpha=0.5)
plt.hist(projected[y[:,0] == 1], bins=10, alpha=0.5)
plt.grid()
plt.xlabel('projections')
plt.ylabel('frequency')
# plt.savefig('figures/flda_twospirals_hist.pdf', bbox_inches="tight")

In [ ]:
# PLOT DECISION BOUNDARY
num=100
points = np.linspace(0, 1, num=num)
A_grid = np.zeros((num*num, 2))
for i in range(num):
    for j in range(num):
        A_grid[i*num+j, 0] = points[i]
        A_grid[i*num+j, 1] = points[j]

A_grid = np.concatenate((A_grid, np.ones((A_grid.shape[0],1))), axis=1) ###

A_grid_projected = np.matmul(A_grid, a)

_,ypred = compute_acc_binary(projected, y, A_grid_projected, np.zeros((A_grid_projected.shape[0],1)), return_pred=True)

In [ ]:
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(6, 6)

plt.rcParams.update({'font.size': 18})

colors = ['r', 'b']
for i in range(A_grid_projected.shape[0]):
    plt.plot(A_grid[i,0], A_grid[i,1], colors[ypred.astype(int)[i,0]]+"s")
# plt.savefig('figures/flda_decision_boundary.pdf', bbox_inches="tight")

### ReLU NFDA

In [ ]:
# ReLU lifting
P = 100

unique_sign_patterns, h_vectors = generate_sign_patterns(A, P, verbose=False)

A_lifted = np.zeros((n, P*d))
for j in range(P):
    A_lifted[:, j*d:(j+1)*d] = (unique_sign_patterns[j] * A.T).T
    
print(A_lifted.shape)

In [ ]:
# solve
class_means, covariances = compute_stats(A_lifted, y, 2)
print(class_means.shape, covariances.shape)

beta = 1
a = np.matmul(np.linalg.inv(covariances[:,:,0] + covariances[:,:,1] + beta*np.eye(class_means.shape[0])), class_means[:,0:1]-class_means[:,1:2])

In [ ]:
# training set
projected = np.matmul(A_lifted, a)
plt.plot(projected[y[:,0] == 0], np.zeros(np.sum(y[:,0] == 0)), "bo")
plt.plot(projected[y[:,0] == 1], np.ones(np.sum(y[:,0] == 1)), "r*")
plt.ylim(-1,2)
plt.grid()
# plt.savefig('figures/fisher_LDA_neural.eps', bbox_inches="tight")

In [ ]:
# solving the LP
u = cp.Variable((d,P))
v = cp.Variable((d,P))

constraints = []
constraints += [u - v == a.reshape((P,d)).T]

for j in range(P):
    constraints += [cp.matmul(((2*unique_sign_patterns[j] - 1) * A.T).T, u[:, j:j+1]) >= 0]
    constraints += [cp.matmul(((2*unique_sign_patterns[j] - 1) * A.T).T, v[:, j:j+1]) >= 0]
    
# Define and solve the CVXPY problem.
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize(0), constraints)
# prob = cp.Problem(cp.Minimize(cp.sum_squares(u)+cp.sum_squares(v)), constraints)
prob.solve(solver=cp.MOSEK)

In [ ]:
# obtain neural network weights
first_layer_weights = np.concatenate((u.value, v.value), axis=1)
second_layer_weights = np.concatenate((np.ones((1,P)), -np.ones((1,P))), axis=1)
print(first_layer_weights.shape, second_layer_weights.shape)

In [ ]:
# forward prop
relu_projected = np.matmul(np.maximum(np.matmul(A, first_layer_weights), 0), second_layer_weights.T)

In [ ]:
# histogram
plt.rcParams.update({'font.size': 18})
plt.hist(relu_projected[y[:,0] == 0], bins=10, alpha=0.5)
plt.hist(relu_projected[y[:,0] == 1], bins=10, alpha=0.5)
plt.grid()
plt.xlabel('projections')
plt.ylabel('frequency')
# plt.savefig('figures/nfda_relu_twospirals_hist.pdf', bbox_inches="tight")

In [ ]:
# PLOT DECISION BOUNDARY
num=100
points = np.linspace(0, 1, num=num)
A_grid = np.zeros((num*num, 2))
for i in range(num):
    for j in range(num):
        A_grid[i*num+j, 0] = points[i]
        A_grid[i*num+j, 1] = points[j]
        
A_grid = np.concatenate((A_grid, np.ones((A_grid.shape[0],1))), axis=1) ###

A_grid_projected = np.matmul(np.maximum(np.matmul(A_grid, first_layer_weights), 0), second_layer_weights.T)

_,ypred = compute_acc_binary(projected, y, A_grid_projected, np.zeros((A_grid_projected.shape[0],1)), return_pred=True)

In [ ]:
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(6, 6)

plt.rcParams.update({'font.size': 18})

colors = ['r', 'b']
for i in range(A_grid_projected.shape[0]):
    plt.plot(A_grid[i,0], A_grid[i,1], colors[ypred.astype(int)[i,0]]+"s")
# plt.savefig('figures/nfda_decision_boundary_beta_{}.pdf'.format(beta), bbox_inches="tight")